In [2]:
import sys
sys.path.append('../')
import config
import tweepy
import os
import json
import codecs

import pandas as pd
import datetime
from datetime import datetime, timedelta
import locale

In [3]:
# Clientを返す認証関数
def auth():
    # API KEY
    CK = config.CK
    CS = config.CS
    AT = config.AT
    AS = config.AS
    BT = config.BT

    # tweepy setting
    # auth = tweepy.OAuthHandler(CK, CS)
    # auth.set_access_token(AT, AS)
    

    # wait_on_rate_limit = True とするとレートリミットを確認しながら取得処理を行う
    client = tweepy.Client(bearer_token = BT,\
                           consumer_key = CK,\
                           consumer_secret = CS,\
                           access_token = AT,\
                           access_token_secret = AS,\
                           wait_on_rate_limit = True)

    return client

In [4]:
# Use tweepy.Client()
client = auth()

In [15]:
# Define a search_term for searching tweets
search_term = "CVE"
# 取得したいツイート数
limit = 50000

In [17]:
# https://qiita.com/daisuke-aoki/items/30e3d6e84637326ef1da
# 取得対象のツイートの時間幅を指定する この例では実行前日と前々日の48時間
# iso形式のUTC時間で指定しないと正しく時間指定ができない模様。
# 指定した時間幅に、limitで指定した件数以上のツイートがあってもlimit以上は取得しない
now = datetime.now()
print(now)
start_time = now - timedelta(days=2)
start_time = start_time.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(start_time)
end_time = now.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(end_time)
# now = now.replace(minute=0, second=0, microsecond=0)
end_time_tweepy = str(end_time.isoformat()) +'+09:00'
# start_time = now - timedelta(days=7) 
start_time_tweepy = str(start_time.isoformat())+'+09:00'
print(start_time_tweepy)
print(end_time_tweepy)

2023-02-21 08:37:45.549600
2023-02-19 00:00:00
2023-02-21 00:00:00
2023-02-19T00:00:00+09:00
2023-02-21T00:00:00+09:00


In [18]:
import numpy as np
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['col_0', 'col_1', 'col_2', 'col_3'],
                  index=['row_0', 'row_1', 'row_2'])
print(type(df))
print(df)

<class 'pandas.core.frame.DataFrame'>
       col_0  col_1  col_2  col_3
row_0      0      1      2      3
row_1      4      5      6      7
row_2      8      9     10     11


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, row_0 to row_2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   col_0   3 non-null      int64
 1   col_1   3 non-null      int64
 2   col_2   3 non-null      int64
 3   col_3   3 non-null      int64
dtypes: int64(4)
memory usage: 120.0+ bytes


In [20]:
df.shape

(3, 4)

In [27]:
df_tweet = pd.DataFrame()
for tweet in tweepy.Paginator(client.search_recent_tweets,\
                              query = search_term,\
                              start_time=start_time_tweepy,\
                              end_time=end_time_tweepy,\
                              tweet_fields=['id','created_at','text','author_id','public_metrics',],\
                              max_results = 100).flatten(limit = limit):
    # print(type(tweet.author_id))
    # print(tweet.author_id)
    # print(dir(tweet.author_id))
    # print(type(tweet.data))
    # print(tweet.data)
    # print(dir(tweet.data))
    # print(pd.DataFrame({'id': pd.Series([tweet.id]),\
    #               'created_at':pd.Series([tweet.created_at]),\
    #               'author_id':pd.Series([tweet.author_id]),\
    #               'public_metrics':pd.Series([tweet.public_metrics]),\
    #               'text':pd.Series([tweet.text]),\
    #               'edit_history_tweet_ids':pd.Series([tweet.edit_history_tweet_ids])}\
    #                    ).head)
    df_tweet = pd.concat([df_tweet, pd.DataFrame({'created_at': pd.Series([tweet.created_at]),\
                                                  'id': pd.Series([tweet.id]),\
                                                  'text': pd.Series([tweet.text]),\
                                                  'author_id': pd.Series([tweet.author_id]),\
                                                  'edit_history_tweet_ids': pd.Series([tweet.edit_history_tweet_ids]),\
                                                  'public_metrics': pd.Series([tweet.public_metrics])\
                                                 },\
                                                )])
    #                                             columns = ['id', 'created_at', 'author_id', 'public_metrics', 'text', 'edit_history_tweet_ids'])\
    #                      ])

df_tweet.head()

,created_at,id,text,author_id,edit_history_tweet_ids,public_metrics
0,2023-02-20 14:59:15+00:00,1627684312001781763,RT @Dinosn: CVE-2022-27677: AMD Ryzen Master P...,1563876813503434752,[1627684312001781763],"{'retweet_count': 17, 'reply_count': 0, 'like_..."
0,2023-02-20 14:58:55+00:00,1627684229315448834,$cve *Top analyst target price for next week &...,1540283817176727552,[1627684229315448834],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-20 14:57:31+00:00,1627683876423274496,"RT @CyberWarship: "" CVSS Scores &amp; Vulnerab...",1563876813503434752,[1627683876423274496],"{'retweet_count': 4, 'reply_count': 0, 'like_c..."
0,2023-02-20 14:51:38+00:00,1627682392965251074,RT @Haqiqatjou: Here is Roberts co writing an ...,1480099215644581889,[1627682392965251074],"{'retweet_count': 24, 'reply_count': 0, 'like_..."
0,2023-02-20 14:50:38+00:00,1627682141696888832,RT @solminingpunk: Some of the notable flaws r...,1458981231903887360,[1627682141696888832],"{'retweet_count': 2, 'reply_count': 0, 'like_c..."


In [28]:
df_tweet.tail()

,created_at,id,text,author_id,edit_history_tweet_ids,public_metrics
0,2023-02-18 15:02:56+00:00,1626960464923209728,CVE-2022-39952 https://t.co/9uc2QYP28H #Harsia...,1083500716235223040,[1626960464923209728],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-18 15:02:21+00:00,1626960314318340096,RT @CVEtrends: Top 3 trending CVEs on Twitter\...,4682904606,[1626960314318340096],"{'retweet_count': 5, 'reply_count': 0, 'like_c..."
0,2023-02-18 15:02:14+00:00,1626960288716308485,"RT @Horizon3Attack: CVE-2022-39952, announced ...",1445960112737165313,[1626960288716308485],"{'retweet_count': 380, 'reply_count': 0, 'like..."
0,2023-02-18 15:01:56+00:00,1626960210937126912,CVE-2023-20032 https://t.co/8F9EvTp10P #Harsia...,1083500716235223040,[1626960210937126912],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-18 15:01:43+00:00,1626960156817752069,"RT @Horizon3Attack: CVE-2022-39952, announced ...",1362016765,[1626960156817752069],"{'retweet_count': 380, 'reply_count': 0, 'like..."


In [29]:
filename = start_time.strftime("%Y%m%d") + "-" + end_time.strftime("%Y%m%d") 
print(filename)

20230219-20230221


In [30]:
df_tweet.to_csv(filename + ".csv", index=False,header=True)